In [53]:
%load_ext autoreload
%autoreload 2
from expressiveness_benchmark.types import Program, Task
import pandas as pd
from dataclasses import replace

from code_widget.example import CodeWidget
import json

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [54]:
# CHANGE ME!
TASK_ID = 'scc'
AUTHOR = 'scott'

In [99]:
task = Task(
    id=TASK_ID,
    name="strongly-connected components",
    description="\
A path is a sequence of edges such that the target of each edge is the source of the next. \
Given a graph and a vertex v, list each vertex u such that u is reachable from v and v is reachable from u.",
    category="Graph Reachability",
    plan=[       
    {
        "id": "graph",
        "description": "a graph",
    }, {
        "id": "source<->target",
        "description": "u is reachable from v and v is reachable from u",
    }, {
        "id": "edge_match",
        "description": "target of each edge is the source of the next",
    }, {
        "id": "edge_sequence",
        "description": "a sequence of edges",
    }, {
        "id": "vertices",
        "description": "the vertices reachable by a path",
    }
    ],
    sample_input={
        "graph": [
            {"source": "a", "target": "b"},
            {"source": "b", "target": "b"},
            {"source": "b", "target": "c"},
            {"source": "c", "target": "b"},
            {"source": "c", "target": "a"},
            {"source": "a", "target": "d"},
        ],
        "query": [{"source": "c"}]
    },
    sample_output=["a", "b", "c"],
)
task.save()

prototype = Program(
    task=TASK_ID,
    author=AUTHOR,
    language=''    
)

In [100]:
datalog = replace(prototype,
    language='datalog',
    source='''.decl scc1(x: symbol, y: symbol)
.decl path(x: symbol, y: symbol)
.decl vertex(x: symbol)

scc(x) :- query(src), scc1(src, x).
scc1(x, x) :- vertex(x).
scc1(x, y) :- path(x, y), path(y, x).
path(x, y) :- graph(x, y).
path(x,y) :- graph(x, z), path(z, y).
vertex(x)  :- graph(x, _).
vertex(x)  :- graph(_, x).''').load_plan()
datalog.execute(task)
datalog.save()

In [101]:
sql = replace(prototype,
    language='sql',
    source='''WITH RECURSIVE
closure(source, target) AS (
  SELECT DISTINCT source, target FROM graph
  UNION
  SELECT edge.source, path.target
  FROM closure as path JOIN graph as edge
  ON edge.target = path.source
),
component(v1, v2) AS (
  SELECT path.source, path.target
  FROM closure as path JOIN closure as path_
  ON path.source = path_.target AND path.target = path_.source
)
SELECT S.v2 FROM component as S
JOIN query ON S.v1 = query.source''').load_plan()
sql.execute(task)
sql.save()

In [102]:
python_imp = replace(prototype,
    language='python-imperative',
    implementation='',
    source='''def scc(graph, query):
  reachable = set()
  vertices = set()
  for edge in graph:
    reachable.add((edge["source"], edge["target"]))
    vertices.add(edge["source"])
    vertices.add(edge["target"])
  changed = True
  while changed:
    changed = False
    for edge in graph:
      for vertex in vertices:
        if ((edge["source"], vertex) not in reachable
            and (edge["target"], vertex) in reachable):
          changed = True
          reachable.add((edge["source"], vertex))
  source = query[0]["source"]
  result = []
  for vertex in vertices:
    if ((source, vertex) in reachable and
        (vertex, source) in reachable):
      result.append(vertex)
  return result''')
python_imp.execute(task)
python_imp.save()

In [103]:
python_fun = replace(prototype,
    language='python-functional',
    implementation='',
    source='''def scc(graph, query):
    def step(relation):
        return set([
            (source, edge["target"])
            for (source, target) in relation
            for edge in graph
            if target == edge["source"]
        ]).union(relation)

    def fix(f, x):
        next = f(x)
        return x if next == x else fix(f, next)

    source = query[0]["source"]
    init = set([(edge["source"], edge["target"]) for edge in graph])
    reachable = fix(step, init)
    return list(set([v for (_, v) in reachable
      if (v, source) in reachable and (source, v) in reachable]))''')
python_fun.execute(task)
python_fun.save()

In [93]:
sql.widget(task)

Output()

CodeWidget(program='{"task": "scc", "language": "sql", "plan": {}, "source": "WITH RECURSIVE\\nclosure(source,…

In [104]:
python_imp.widget(task)

Output()

CodeWidget(program='{"task": "scc", "language": "python-imperative", "plan": {}, "source": "def scc(graph, que…

In [93]:
sql.widget(task)

Output()

CodeWidget(program='{"task": "scc", "language": "sql", "plan": {}, "source": "WITH RECURSIVE\\nclosure(source,…

In [62]:
# todo?
pandas = replace(prototype,
    language='python-pandas',
    implementation='',
    source='''def scc(graph):
    
''')
pandas.execute(task)
pandas.save()

SyntaxError: unexpected EOF while parsing (<string>, line 6)